In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
WIDTH = 100
HEIGHT = 50
FOV = 90
FOV_RATIO = HEIGHT/WIDTH

In [ ]:
CAMERA_POSITION = np.array([0, 0, 4])
CAMERA_UP = np.array([0, 0, 1])
CAMERA_FORWARD = np.array([1, 0, 0])
CAMERA_RIGHT = np.cross(CAMERA_UP, CAMERA_FORWARD)
CAMERA_TRANSFORM = np.matrix([CAMERA_FORWARD, CAMERA_RIGHT, CAMERA_UP])

In [ ]:
def normalize(v):
    return v / np.linalg.norm(v)

def proj(u, v):
    return np.dot(u, v) * normalize(v)

def reflect(v, n):
    return v - (np.dot(2*v, n) * n)

In [ ]:
class Material:
    def __init__(self, ambient, diffuse, spectral, spectral_p):
        self.ambient = np.array(ambient)
        self.diffuse = np.array(diffuse)
        self.spectral = np.array(spectral)
        self.spectral_p = spectral_p
        
    def get_ambient_coef(self):
        return self.ambient
    
    def get_diffuse_coef(self):
        return self.diffuse
    
    def get_spectral_coef(self):
        return self.spectral
    
    def get_spectral_p_coef(self):
        return self.spectral_p

In [ ]:
class Sphere:
    def __init__(self, position, radius, material):
        self.position = position
        self.radius = radius
        self.material = material
        
    def get_distance(self, location):
        return np.linalg.norm(location-self.position) - self.radius
    
    def get_surface_normal(self, location):
         return normalize(location-self.position)
     
class Plane:
    def __init__(self, position, normal, material):
        self.position = position
        self.normal = normalize(normal)
        self.material = material
        
    def get_distance(self, location):
        return np.linalg.norm(proj((location - self.position), self.normal))
    
    def get_surface_normal(self, location):
        return self.normal

In [ ]:
class Light:
    def get_ray(self, location=None):
        return (0, 0, 0)
        
class DirectionalLight(Light):
    def __init__(self, direction, color):
        self.direction = normalize(direction)
        self.color = color
        
    def get_ray(self, location=None):
        return self.direction
        
class PointLight(Light):
    def __init__(self, position, color):
        self.position = position
        self.color = color
        
    def get_ray(self, location):
        return normalize(self.position - location)

In [ ]:
OBJECTS = [
    Sphere(np.array([15, -8, 4]), 4, Material((.5, .02, .02), (0, 0, 0), (0, 0, 0), 1)),
    Sphere(np.array([15, 0, 4]), 4, Material((.02, .2, .02), (0, 1, 0), (.8, .8, .8), .8)),
    Sphere(np.array([15, 8, 4]), 4, Material((.02, .02, .2), (0, 0, 1), (0, 0, 0), 1)),
    Plane(np.array([0, 0, 0]), np.array([0.01, 0, 1]), Material((0, 0, 0), (.5, .5, .5), (0, 0, 0), 1)),
    Plane(np.array([20, 0, 0]), np.array([-1, 0, 0]), Material((0, 0, 0), (.5, .5, .5), (0, 0, 0), 1))
]

LIGHTS = [
    DirectionalLight((.5, 0, -1), (1, 1, 1))
]


In [ ]:
class Ray:
    def __init__(self, ray_position, ray_direction):
        self.ray_position = ray_position
        self.ray_direction = ray_direction
        self.time = 1
    
    def get_next_pos(self, distance):
        self.time += distance
        return (self.time * self.ray_direction) + self.ray_position

In [ ]:
def get_pixel_color(x, y):
    # what is the ray coming from this pixel
    theta = np.radians(((x - (WIDTH/2)) / WIDTH) * FOV)
    rho =  np.radians(((y - (HEIGHT/2)) / HEIGHT) * FOV * FOV_RATIO)
    x_rotation = np.matrix([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta),  np.cos(theta), 0],
        [0            ,  0            , 1]
    ])
    y_rotation = np.matrix([
        [ np.cos(rho), 0, np.sin(rho)],
        [ 0          , 1, 0          ],
        [-np.sin(rho), 0, np.cos(rho)]
    ])
    direction_transform = CAMERA_TRANSFORM * x_rotation * y_rotation * np.linalg.inv(CAMERA_TRANSFORM)
    ray_direction = np.resize(direction_transform @ CAMERA_FORWARD, (3,))
    camera_ray = Ray(CAMERA_POSITION, ray_direction)
    
    # Does this ray hit anything
    object_hit = None
    hit_location = None
    
    current_ray_traveled = 0
    for _ in range(1000):
        current_ray = current_ray_traveled*camera_ray.ray_direction + camera_ray.ray_position
        
        min_distance = float('inf')
        min_object = -1
        for i, object in enumerate(OBJECTS):
            this_distance = object.get_distance(current_ray)
            if this_distance < min_distance:
                min_distance = this_distance
                min_object = i
                
        if min_distance < 0.001:
            object_hit = OBJECTS[min_object]
            hit_location = current_ray
            break
        elif min_distance > 1000:
            break
        else:
            current_ray_traveled += min_distance
            
    pixel_color = np.array((0.0, 0.0, 0.0), dtype=np.float64)
    
    if object_hit is not None:
                
        # Ambient Shading
        pixel_color += object_hit.material.get_ambient_coef()
        
        # Diffuse Shading
        diffuse_color = np.array([0.0, 0.0, 0.0])
        for light in LIGHTS:
            brightness = max(np.dot(light.get_ray(hit_location), -normalize(object_hit.get_surface_normal(hit_location))), 0)
            diffuse_color += object_hit.material.get_diffuse_coef() * (brightness * np.array(light.color))
        pixel_color += diffuse_color
        
        # Spectral Shading
        spectral_color = np.array([0.0, 0.0, 0.0])
        for light in LIGHTS:
            brightness = max(-(normalize(reflect(light.get_ray(hit_location), object_hit.get_surface_normal(hit_location))) @ camera_ray.ray_direction),0) ** object_hit.material.get_spectral_p_coef()
            spectral_color += object_hit.material.get_spectral_coef() * (brightness * np.array(light.color))
        pixel_color += spectral_color
        
    return pixel_color

In [ ]:
# image = [[0 for _ in range(WIDTH)] for _ in range(HEIGHT)]
# def set_pixel(x, y):
#     image[y][x] = get_pixel_color(x, y)

from multiprocessing import Pool

pixel_coords = [(x, y) for x in range(WIDTH) for y in range(HEIGHT)]
def test(x):
    print(x)
    return x*x
with Pool(1) as pool:
    # image = pool.map(get_pixel_color, pixel_coords)
    print(pool.map(test, [1, 2, 3, 4]))
    pool.close()
    pool.join()
# for x, y in pixel_coords:
#     set_pixel(x, y)
        
# SATURATION = 1.5
# image = (np.array(image) / (np.max(image)/SATURATION)).clip(0, 1)

In [ ]:
print(image.shape)
plt.imshow(image)

In [ ]:
plt.imsave("output.png", image)